In [ ]:
!pip install --upgrade scikit-learn==0.20.3

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn import neighbors
from sklearn.metrics import mean_squared_error
import math
from sklearn.metrics import accuracy_score
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO 
from IPython.display import Image 
from pydot import graph_from_dot_data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/corona_tested_individuals_ver_0083.english.csv")
df

In [ ]:
df.dropna(inplace = True)
df['test_date']=pd.DatetimeIndex(df['test_date'])
df = df.set_index('test_date')

In [ ]:
sns.countplot(df['corona_result'])


In [ ]:
co = df.copy(True)
co['gender'] = co['gender'].map({'female': 0, 'male': 1})
# co['test_indication'] = co['test_indication'].map({'Other': 0, 'Contact with confirmed': 1 ,'Abroad':2 })
co = co.loc[co.test_indication == 'Contact with confirmed']
co = co[co.age_60_and_above != 'NaN']
co['age_60_and_above'] = co['age_60_and_above'].map({'Yes': 1,'No':0 })
co = co[co.corona_result != 'other']
co['corona_result'] = co['corona_result'].map({'positive': 1, 'negative' : 0  })





In [ ]:
co.corona_result.unique()

In [ ]:
co

In [ ]:
del co['test_indication']


In [ ]:
co.describe()

In [ ]:
co

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances




In [ ]:
jac_co = co.to_numpy()


In [ ]:
jc= pairwise_distances(jac_co, metric = "jaccard")
print(jc)
jh =pairwise_distances(jac_co, metric = "hamming")

In [ ]:
djc = pd.DataFrame(jc, index=co.T.columns, columns=co.T.columns)
djh = pd.DataFrame(jh, index=co.T.columns, columns=co.T.columns)

In [ ]:
djc

In [ ]:
djh

In [ ]:
ax = sns.heatmap(co, vmin=0, vmax=1)


In [ ]:
y  = co.corona_result.values
X = co.loc[:, co.columns != 'corona_result'].values


In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Fitting classifier to the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 2)
classifier.fit(X_train, y_train)

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
ac1 = accuracy_score(y_pred,y_test)  
ac1

In [ ]:
error = []

# Calculating error for K values between 1 and 20
for i in range(1, 20):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred_i = knn.predict(X_test)
    error.append(np.mean(pred_i != y_test))

In [ ]:
error

In [ ]:
plt.figure(figsize=(3, 6))
plt.plot(range(1, 20), error, color='red', linestyle='dashed', marker='o',
         markerfacecolor='blue', markersize=10)
plt.title('Error Rate K Value')
plt.xlabel('K Value')
plt.ylabel('Mean Error')

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)


In [ ]:
dot_data = StringIO()
export_graphviz(dt, out_file=dot_data, feature_names=co.loc[:, co.columns != 'corona_result'].columns)
(graph, ) = graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

In [ ]:
y_pred = dt.predict(X_test)


In [ ]:
ac2 = accuracy_score(y_pred,y_test)  
ac2